# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- template script for analyzing modes spectra for sumatra
- downloads LHZ data needed for hw_sumatraB

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import warnings

from obspy import read
from pysep import Pysep

from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from lib_seis import sumatra_waveform_screening
from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake <br/>
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for item in event:
    print(f'{item} = {event[item]}')
print()    

### PySEP - Python Seismogram Extraction and Processing

- We will gather data for this event using the [PySEP](https://pysep.readthedocs.io/en/devel/index.html)
package.
- The PySEP package uses [ObsPy](https://docs.obspy.org/) internally to collect and handle seismic data.
- Check the webpages for details on
[data gathering](https://pysep.readthedocs.io/en/devel/autoapi/pysep/pysep/index.html#pysep.pysep.Pysep) and
[record section plotting](https://pysep.readthedocs.io/en/devel/autoapi/pysep/recsec/index.html#pysep.recsec.RecordSection)
- Check the (IRIS)[https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming] webpage for the SEED
format seismic channel naming

In [ ]:
# raw LHZ data download specifications

stations   = dict( networks                     = 'G,II,IU',
                   stations                     = '*',
                   locations                    = '*',
                   channels                     = 'LHZ' )

screening  = dict( seconds_before_ref           =  0.5 * 24 * 60 * 60,
                   seconds_after_ref            = 10.0 * 24 * 60 * 60,
                   remove_clipped               = False,
                   remove_insufficient_length   = False,
                   fill_data_gaps               = 0,
                   gap_fraction                 = 1,
                   resample_freq                = 1     )

processing = dict( remove_response              = False ) 
                   
output     = dict( log_level                    = 'INFO',
                   plot_files                   = 'map',
                   output_dir                   = 'datawf',
                   sac_subdir                   = '', 
                   overwrite_event_tag          = 'sumatra_LHZ_raw',
                   overwrite                    = False )

In [ ]:
# download data

data_dir  = f'{output["output_dir"]}/{output["overwrite_event_tag"]}'
overwrite = f'{output["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**stations,**screening,**processing,**output)
    ps.run()
else:
    print('\nData directory already exists, no data will be downloaded\n')

In [ ]:
# read downloaded raw LHZ data

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# waveforms to be rejected outright

waveforms_to_reject, waveforms_to_trim = sumatra_waveform_screening(channel='LHZ')

# adding waveforms to trim to the list of waveforms to reject
for item in waveforms_to_trim:
    waveforms_to_reject.append(item[0])

# sorting the list alphabetically
waveforms_to_reject = sorted(waveforms_to_reject)    
    
# listing waveforms IDs to be rejected outright
print('List of waveforms to reject outright - \n')
print(f'{len(waveforms_to_reject)} waveforms:')
for waveform_id in waveforms_to_reject:
    print(waveform_id)
print()

In [ ]:
# plotting waveforms to be rejected outright   
'''
for waveform_id in waveforms_to_reject:
    st_subset = st.select(id=waveform_id)
    plt.figure()
    # replacing filled gaps (0's) with gaps (Nan's)
    plt.plot(np.where(st_subset[0].data==0.00000000, np.nan, st_subset[0].data))
    plt.xlim(0,907200)
    plt.title(f'{st_subset[0].id} − starting {st_subset[0].stats["starttime"]}')
'''

In [ ]:
# waveforms retained

retained_waveforms = []
for tr in st:
    if tr.id not in waveforms_to_reject:
        retained_waveforms.append(tr.id)

In [ ]:
# plotting and listing station information corresponding to retained waveforms

event_path = f'{data_dir}/event.xml'
inv_path = f'{data_dir}/inv.xml'
station_map_and_table(event_path, inv_path, subset_ids=retained_waveforms)

In [ ]:
# computing and plotting the amplitude spectra of the retained waveforms
'''
if True:

    # resample_freq is provided as input into pysep (see above)
    Nyquist_frequency = screening['resample_freq']/2

    for waveform_id in retained_waveforms:

        st_subset = st.select(id=waveform_id).copy()

        st_subset.detrend('constant')
        st_subset.detrend('linear')
        st_subset.taper(max_percentage=0.2, type="cosine")

        fft_amp, fft_phase, fft_freq = wf_fft(st_subset[0].data, Nyquist_frequency)

        plt.figure()
        plt.plot(fft_freq*1E3,fft_amp)
        plt.xlim(0.2,1)
        #plt.ylim(0,40)
        xmask = np.logical_and(fft_freq*1e3 > .2, fft_freq*1e3 < 1)
        plt.ylim(0,np.max(fft_amp[xmask])*1.1)
        plt.title(st_subset[0].id)
        plt.xlabel(f'Frequency (mHz)')
        plt.ylabel(f'Amplitude (counts s)')
        plt.show()        
'''

In [ ]:
# pick at least 20 waveforms having a useful and clean amplitude spectrum
# pick only one location for a given station

# list of stations with sensors at multiple locations
# network = G  - KIP
# network = II - ALE, ASCN, BORG, DGAR, KDAK, PFO, RPN, WRAB
# network = IU - AFI, ANMO, CASY, COLA, FURI, GUMO, HNR, INCN, KIP, QSPA
#                RAO, RAR, RCBR, SAML, SDV, SNZO, TATO, TEIG, TRIS, TRQA, WAKE

# waveform = 'network.station.location.channel'

picked_waveforms = ['G.CAN..LHZ'    ,
                    'II.NNA.00.LHZ' ,
                    'II.SUR.00.LHZ' ,
                    'IU.BBSR.00.LHZ']

In [ ]:
# plotting and listing station information corresponding to selected waveforms

event_path = f'{data_dir}/event.xml'
inv_path = f'{data_dir}/inv.xml'
station_map_and_table(event_path, inv_path, subset_ids=picked_waveforms)

In [ ]:
# response removed LHZ data download for selected station locations
# response removed to acceleration
    
stations   = dict( station_ids                  = picked_waveforms )

screening  = dict( seconds_before_ref           =  0.5 * 24 * 60 * 60,
                   seconds_after_ref            = 10.0 * 24 * 60 * 60,
                   remove_clipped               = False,
                   remove_insufficient_length   = False,
                   fill_data_gaps               = 0,
                   gap_fraction                 = 0.03,
                   resample_freq                = 1           )

processing = dict( remove_response              = True,
                   output_unit                  = 'ACC',                                   
                   water_level                  = None,
                   pre_filt                     = 'default',
                   detrend                      = True,
                   demean                       = True,
                   taper_percentage             = 0.2         ) 

output     = dict( log_level                    = 'DEBUG',
                   plot_files                   = 'map',
                   output_dir                   = 'datawf',
                   sac_subdir                   = '',
                   overwrite_event_tag          = 'sumatra_LHZ_acc',
                   overwrite                    = True )

In [ ]:
# download data

data_dir  = f'{output["output_dir"]}/{output["overwrite_event_tag"]}'
overwrite = f'{output["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**stations,**screening,**processing,**output)
    ps.run()
else:
    print('Data directory already exists, no data will be downloaded')

In [ ]:
# read downloaded LHZ acceleration data for the selected station locations

st_acc = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded LHZ waveforms with response removed to acceleration for the selected station '
      'locations - \n')
print(st_acc.__str__(extended=True))
print()

In [ ]:
# computing and plotting the amplitude spectra of the LHZ acceleration waveforms for the selected station
# locations

if True:

    # resample_freq is provided as input into pysep (see above)
    Nyquist_frequency = screening['resample_freq']/2

    for tr_acc in st_acc:

        tr = tr_acc.copy()
        
        tr.detrend('constant')
        tr.detrend('linear')
        tr.taper(max_percentage=0.2, type="cosine")

        fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

        plt.figure()
        plt.plot(fft_freq*1E3,fft_amp)
        plt.xlim(0.2,1)
        #plt.ylim(0,40)
        xmask = np.logical_and(fft_freq*1e3 > .2, fft_freq*1e3 < 1)
        plt.ylim(0,np.max(fft_amp[xmask])*1.1)
        plt.title(f'{tr.id}')
        plt.xlabel(f'Frequency (mHz)')
        plt.ylabel(f'Amplitude (ms^-2 s)')
        plt.show()

